In [ ]:
import os
import pandas as pd
import pyvista as pv
from core.project import Project
from striplog import Component
from utils.config import DEFAULT_LITHO_LEXICON
from definitions import ROOT_DIR

## Create a new project from a empty or existing database

In [ ]:
update_db = False
db_name = './new_db.db'
db_bkp = db_name.replace('.db', '')+'_bkp.db'
if os.path.exists(db_name) and not update_db:
    if os.path.exists(db_bkp):
        os.remove(db_bkp)
    os.rename(db_name, db_bkp)

In [ ]:
p = Project.load(db_name)

In [ ]:
data = ROOT_DIR + '/CF_data/Memoris.xlsx'

In [ ]:
bh_df = pd.read_excel(data, sheet_name='boreholes')
bh_df['ID'] = bh_df['ID'].astype('str')
bh_df.head()

In [ ]:
for idx, row in bh_df.iterrows():
    bh_dict = {'id':row['ID'], 'borehole_type': row['Type'], 'length': row['Length'], 'diameter':row['Diameter']/1000.}
    if not pd.isnull(row['Date'].date()):
        bh_dict.update({'date': row['Date'].date()})
    bh_dict.update({'top':{'middle': row['Z'], 'x': row['X'], 'y': row['Y']}, 'base':{'middle': row['Z']-row['Length'], 'x': row['X'], 'y': row['Y']}})
    p.add_borehole(bh_dict)

In [ ]:
intv_df = pd.read_excel(data, sheet_name='intervals')
intv_df['ID'] = intv_df['ID'].astype('str')
intv_df.head()

In [ ]:
intv_df = intv_df.merge(bh_df, how='left', left_on='ID', right_on='ID')[['ID', 'De', 'A', 'Keyword', 'Description_x', 'X', 'Y', 'Z']]
intv_df.head()

In [ ]:
intv_df['top'] = intv_df['Z'] - intv_df['De']
intv_df['base'] = intv_df['Z'] - intv_df['A']
intv_df.rename({'Description_x': 'Description'}, axis=1, inplace=True)
intv_df.head()

In [ ]:
bh = ''
intv_nr = 0
for idx, row in intv_df.iterrows():
    if row.ID!=bh:
        bh=row.ID
        intv_nr=0
    else:
        intv_nr+=1
    components = Component.from_text(row['Keyword'].replace('emblais', 'emblai'), DEFAULT_LITHO_LEXICON, first_only=True).__dict__
    if len(components)==0:
        components=[]
        print(f'Could not derive a component from {row["Keyword"]} in interval {intv_nr} of borehole {row.ID}.')
    else:
        components= [str(components)]
    intv_dict={'description': row['Description'],
           'top':{'middle': row.top, 'x': row.X, 'y': row.Y},
           'base':{'middle': row.base, 'x': row.X, 'y': row.Y},
           'interval_number': intv_nr,
           'components': components,
           'extra_data': None
          }
    p.insert_interval_in_borehole(row.ID, intv_dict)

In [ ]:
p.repr_attribute='lithology'

In [ ]:
p.refresh(update_3d=True)

### Visualization

In [ ]:
bh_name = '201'

In [ ]:
p.boreholes_3d[bh_name].intervals

In [ ]:
sel_intervals = p.boreholes_orm[bh_name].intervals
print(len(sel_intervals.keys()), 'intervals')
sel_intervals[list(sel_intervals.keys())[1]]

In [ ]:
p.plot_log(bh_name, ticks=(.5, 2))

### add ground surface

In [ ]:
ground_surface = pv.read('ground_surface.vtk')
#container = pv.cube(center=(0.0, 0.0, 0.0), x_length=5.92, y_length=2.35, z_length=2.35)

In [ ]:
p.repr_attribute= 'lithology' # 'borehole_type'

In [ ]:
pl = pv.Plotter(window_size=(800,600), notebook=False)
pl.add_mesh(p.vtk, cmap=p.cmap, scalar_bar_args=p.scalar_bar_args, show_scalar_bar=True, annotations=p.annotations)
pl.add_mesh(ground_surface, color='green', opacity=0.5 )
pl.add_axes()
pl.show()

In [ ]:
p.boreholes_3d[bh_name]._vtk

In [ ]:
p.repr_attribute

In [ ]:
p.boreholes_3d[bh_name].legend_dict

In [ ]:
p.plot_3d(window_size=(800, 600), str_annotations=True, verbose=True)

# Tests

In [ ]:
p.legend_dict['borehole_type']['legend']

In [ ]:
pl = pv.Plotter(window_size=(800,600))

In [ ]:
p.boreholes_3d['201'].plot_3d(plotter=pl, repr_attribute='borehole_type', repr_legend_dict=p.legend_dict, 
                         repr_cmap=p.legend_dict['borehole_type']['cmap'],
                         repr_uniq_val=p.legend_dict['borehole_type']['values'])

In [ ]:
p.legend_dict['borehole_type']['cmap']

In [ ]:
p.boreholes_3d['201'].vtk()

In [ ]:
pl.add_mesh(p.boreholes_3d['201']._vtk)

In [ ]:
pl.show(jupyter_backend=False)

In [ ]:
from utils.orm import get_interval_list

In [ ]:
get_interval_list(p.boreholes_orm['201'], 'borehole_type')[0][0].top.z

In [ ]:
p.boreholes_orm['201'].id